In [20]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime
import helpers

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [21]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


2

## Prepare cost and prediction functions

In [51]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    N = y.shape[0]
    z = y * (X @ w)
    tmp = 1 - z
    cost = tmp[tmp > 0].sum()
    cost += 0.5 * lambda_ * w.T @ w
    return cost

In [28]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    N = y.shape[0]
    y_pred = helpers.predict_labels(w, X)
    n_correct = (np.abs(y + y_pred) / 2).sum()
    return n_correct / N
    

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [24]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    gradient = None
    x_n, y_n = X[n], y[n]
    z = y_n * x_n.T @ w
    if 1 - z <= 0:
        gradient = lambda_ * w
    else:
        gradient = - y_n * x_n
        gradient += lambda_ * w

    return gradient

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [35]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective_fast(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=37055.16675338409
iteration=10000, cost=458.49380166318844
iteration=20000, cost=453.4664508178323
iteration=30000, cost=453.3877584329211
iteration=40000, cost=452.4780824502913
iteration=50000, cost=453.3361227280477
iteration=60000, cost=453.527338974119
iteration=70000, cost=453.83364629589465
iteration=80000, cost=453.63027587019474
iteration=90000, cost=453.80929801850783
iteration=100000, cost=453.7852309876965
iteration=110000, cost=453.84963580902325
iteration=120000, cost=453.6095889817368
iteration=130000, cost=454.04862782883174
iteration=140000, cost=453.5186629656726
iteration=150000, cost=453.1947773836651
iteration=160000, cost=453.1132291394824
iteration=170000, cost=453.0230884432675
iteration=180000, cost=453.2071059607605
iteration=190000, cost=453.0315876280717
training accuracy = 0.7715289982425307


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [69]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    N = y.shape[0]
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    e_n = np.zeros(N)
    e_n[n] = 1.0

    min_ = -old_alpha_n
    max_ = 1 - old_alpha_n

    # dominant coefficient (2 degree polynomial) < 0
    a = - 1 / ( 2*lambda_ ) * np.linalg.norm(y_n * x_n)

    # second dominant coefficient
    # tmp = y_n * alpha.T @ Y @ (X @ x_n)
    # b = 1 - 1 / lambda_ * tmp
    b = 1 - y_n * w.T @ x_n

    middle = - b / (2 * a)
    
    gamma = 0
    if middle > min_ and middle < max_:
        gamma = middle
    elif middle <= min_:
        gamma = min_
    elif middle >= max_:
        gamma = max_
    else:
        print('NOPE')
        gamma = 0
    
    alpha += e_n * gamma
    # if (alpha < 0).sum() > 0 or (alpha > 1).sum() > 0:
    #     print('DAMNED')

    w += gamma/lambda_ * x_n * y_n
    return w, alpha

In [40]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    N = alpha.shape[0]
    Y = np.diag(y)
    Q = alpha.T @ Y @ X @ X.T @ Y @ alpha
    return alpha.T @ np.ones(N) - 1 / (2 * lambda_) * Q

In [70]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:54602.11985, dual:-211.49854, gap:54813.61840
iteration=10000, primal:10084.40109, dual:-2406.75405, gap:12491.15514
iteration=20000, primal:14119.10646, dual:-1932.41785, gap:16051.52431
iteration=30000, primal:68197.42632, dual:-2816.81679, gap:71014.24311
iteration=40000, primal:24471.19609, dual:-2470.91007, gap:26942.10616
iteration=50000, primal:18556.79195, dual:-2281.83445, gap:20838.62640
iteration=60000, primal:4461.55365, dual:-3172.57129, gap:7634.12494
iteration=70000, primal:40856.74284, dual:-3329.04557, gap:44185.78841


KeyboardInterrupt: 